In [1]:
!node --version

v14.18.0


In [2]:
import sys
sys.path.append("/Users/alexandrec/Kaggle/simple")

import os
os.getcwd() 

'/Users/alexandrec/Kaggle/simple'

In [3]:
!pip install kaggle-environments -U
!pip install torch

You should consider upgrading via the '/Users/alexandrec/Kaggle/project-env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/alexandrec/Kaggle/project-env/bin/python -m pip install --upgrade pip' command.


In [4]:
from kaggle_environments import make
import numpy as np
import torch

Loading environment football failed: No module named 'gfootball'


In [5]:
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2, "annotations": True}, debug=True)


In [6]:
# test agent 
from agent import agent
steps = env.run([agent, agent])
# env.render(mode="ipython", width=1200, height=800)

In [39]:
import math, sys
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate


import torch
import numpy as np

DIRECTIONS = Constants.DIRECTIONS
game_state = None

# 
def agent_train(observation, model, params):
    """This agent is for training the model only, do not submit it"""

    global game_state
    
    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    game_map = game_state.map
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_map.width, game_map.height

    resource_tiles: list[Cell] = []
    city_tiles: list[Cell] = []

    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
            if cell.citytile is not None:
                city_tiles.append(cell)
    
    def get_cell_resources(cell: Cell):
        wood = 0
        coal = 0
        uranium = 0
        if cell.has_resource():
            wood = cell.resource.amount if cell.resource.type == "wood" else 0
            coal = cell.resource.amount if cell.resource.type == 'coal' else 0
            uranium = cell.resource.amount if cell.resource.type == 'uranium' else 0
        return wood, coal, uranium
    
    def get_adjacent_resources(pos):
        wood = 0
        coal = 0
        uranium = 0
        x, y = (pos.x, pos.y)
        for i in range(-1,1):
            for j in range(-1,1):
                cell = game_map.get_cell(x+i,y+j)
                if cell.has_resource():
                    wood += cell.resource.amount if cell.resource.type == "wood" else 0
                    coal += cell.resource.amount if cell.resource.type == 'coal' else 0
                    uranium += cell.resource.amount if cell.resource.type == 'uranium' else 0
        return wood, coal, uranium
    
    def get_units_state():
        units = []
        for unit in player.units:
            if unit.can_act():
                cell = game_map.get_cell_by_pos(unit.pos)
                worker_type = 0 if unit.is_worker() else 1
                cargo = unit.get_cargo_space_left()
                pos_x = unit.pos.x
                pos_y = unit.pos.y
                is_in_city = cell.citytile != None
                adjacent_wood, adjacent_coal, adjacent_uranium = get_adjacent_resources(unit.pos)
                units.append([worker_type, cargo, adjacent_wood, adjacent_coal, adjacent_uranium, is_in_city, pos_x, pos_y])
        return units         
    
    def get_cell_type(cell):
        cell_code = 0
        all_units = [player.units, opponent.units]
        for unit in player.units:
            if unit.pos == cell.pos:
                cell_code = 1
                if unit.is_cart:
                    cell_code +=0.5
        for unit in opponent.units:
            if unit.pos == cell.pos:
                cell_code = 2
                if unit.is_cart:
                    cell_code +=0.5
        return cell_code
                    

    def get_global_state():
        state = []
        for x in range(width):
            for y in range(height):
                cell = game_map.get_cell(x,y)
                w, c, u = get_cell_resources(cell)
                cell_occupant = get_cell_type(cell)
                cityid = int(cell.citytile.cityid[2:])+1 if cell.citytile else 0
                
                city = (cell.citytile.team+1) if cell.citytile else 0
                fuel = opponent.cities[cell.citytile.cityid].fuel if city == opponent.team+1 else player.cities[cell.citytile.cityid].fuel if city == player.team+1 else 0
                road = cell.road
                state.append([w, c, u, cell_occupant, cityid, fuel, city, road, x, y])
        return state
                

    def cell_is_free(cell):
        return cell.citytile is None and not cell.has_resource()
    
    def get_random_actions(unit):
        i = np.randint(9, size=1)[0]
        actions = [unit.move[NORTH], unit.move[SOUTH], unit.move[EAST], unit.move[WEST], 
                   unit.build_city(), unit.pillage()]
        return actions[i]
    
    global_state_tensor = torch.tensor(get_global_state())
    explore = np.random.rand() < params.exploration_rate
    
    # pad if map size < 32
    
    for citytile in city_tiles:
        # city policy here
        pass
    print(player.team)
    for unit in player.units:
        if unit.is_worker() and unit.can_act():
            # EXPLORE
            if (explore):
                actions.append(get_random_actions(unit))
            else:
            # EXPLOIT
                unit_state_tensor = torch.tensor(get_units_state())
                action = model(state_tensor, units_state_tensor, mode="online")
                actions.append(action)
            
            # DEBUG
    


    return actions



In [40]:
import torch
import copy
class MapEncoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(10, 64, 8, 2)
        self.conv2 = torch.nn.Conv2d(64, 128, 4, 2)
        self.relu = torch.nn.ReLU()
        self.ffn_map = torch.nn.Linear(3200,512)
    def forward(self, x):
        y = self.conv1(x)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.relu(y)
        flat = y.view(-1, 3200)
        out = self.ffn_map(flat)
        return out

class UnitRNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn_unit = torch.nn.GRU(8, 128, 2, dropout=0)
    def forward(self, x):
        hn = torch.zeros(2, 1, 128)
        for k in range (x.shape[1]):
            pred, hn = self.rnn_unit(x[:,k,:].unsqueeze(1), hn)
        return pred

class LuxrNet(torch.nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.unitRnn = UnitRNN()
        self.mapEncoder = MapEncoder()
        self.relu = torch.nn.ReLU()
        self.joiner = torch.nn.Linear(640,512)
        self.output = torch.nn.Linear(512,output_dim)

    def forward(self, global_features, units_features):
        glob = self.mapEncoder(global_features)
        units = self.unitRnn(units_features)
        hidden_out = self.joiner(torch.cat((glob, units.squeeze(1)), 1))
        out = self.output(self.relu(hidden_out))
        return out

class DDQN(torch.nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.online = LuxrNet(output_dim)
        self.target = copy.deepcopy(self.online)
        
        for p in self.target.parameters():
            p.requires_grad = False

    def forward(self, global_features, units_features, mode):
        if mode == "online":
            return self.online(global_features, units_features)
        if mode == "target":
            return self.target(global_features, units_features)
        
        
    
    
    
t = torch.rand(2, 10, 32, 32)
# model = MapEncoder()
# model.train()
# out = model(t)

t2 = torch.rand(2, 10, 8)
print(t2[:,1,:].shape)
# rnn = UnitRNN()
# out = rnn(t2)
model = DDQN(10)
y = model(t,t2,"online")
print(y)




torch.Size([2, 8])
tensor([[-0.0031,  0.0014, -0.0110,  0.0371,  0.0047, -0.0483,  0.0280, -0.0665,
          0.0176, -0.0122],
        [-0.0069, -0.0017, -0.0095,  0.0408,  0.0096, -0.0410,  0.0314, -0.0541,
          0.0165, -0.0084]], grad_fn=<AddmmBackward>)


In [ ]:
# optimizer.zero_grad()        
# loss, hidden = model(data, hidden, targets)
# loss.backward()

# torch.nn.utils.clip_grad_norm(model.parameters(), args.clip)
# optimizer.step()

In [28]:
from model import DDQN

class TrainingParameters:
    def __init__(self):
        self.exploration_rate = 1
        self.exploration_rate_decay = 0.99999975
        self.exploration_rate_min = 0.1
        self.curr_step = 0
        self.save_every = 5e5


class Bot:
    def __init__(self, agent, save_dir):
        self.parameters = TrainingParameters()
        self.use_cuda = torch.cuda.is_available()
        self.save_dir = save_dir
        self.action_space = 10
        self.model = DDQN(self.action_space)
        self.agent = agent_train
        if self.use_cuda:
            self.net = self.net.to(device="cuda")

    def play(self,obs):
        """Play 1 step with epsilon-greedy policy"""
        return self.agent(obs, self.model, self.parameters)

    def cache(self, obs, reward, done, info):
        pass
    
    def get_batch(self):
        """Sample a batch from the cache"""
        pass
    def learn(self):
        pass
bot = Bot(agent, "./")

In [ ]:
# training loop with step API (workaround for the reward)
from lux.game import Game
from functools import partial
params = TrainingParameters()
agent = partial(agent_train, params=params)
game_state = Game()
episodes = 1

def update_game_state(observation):
    global game_state
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    return game_state

def get_reward(obs):
    game_state = update_game_state(obs)
    player = game_state.players[0]
    ct_count = sum([len(v.citytiles) for k, v in player.cities.items()])
    unit_count = len(game_state.players[player.team].units)
    return ct_count * 10000 + unit_count

for e in range (episodes):
    print("episode {}".format(e))
    env.reset()
    trainer = env.train([None,agent])
    obs = trainer.reset()
    while True:
        action = agent(obs, "training")
        next_obs, reward, done, info = trainer.step(action)
        print("step")
        # reward workaround
        # reward = get_reward(obs)
        obs = next_obs
        if done:
            break

env.render(mode="ipython", width=1200, height=800)


In [ ]:
batch_size = 5

In [ ]:

# env.render(mode="ipython", width=1200, height=800)

In [ ]:
env.reset()


In [ ]:
import math, sys
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate

import random

DIRECTIONS = Constants.DIRECTIONS
game_state = None


def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    resource_tiles: list[Cell] = []
    city_tiles: list[Cell] = []

    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
            if cell.citytile is not None:
                city_tiles.append(cell)
    
    def cell_is_free(cell):
        return cell.citytile is None and not cell.has_resource()
    
    # we iterate over all our units and do something with them
    for unit in player.units:
        if unit.is_worker() and unit.can_act():
            closest_dist = math.inf
            closest_resource_tile = None
            if unit.get_cargo_space_left() > 0:
                # if the unit is a worker and we have space in cargo, lets find the nearest resource tile and try to mine it
                for resource_tile in resource_tiles:
                    if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
                    if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
                    dist = resource_tile.pos.distance_to(unit.pos)
                    if dist < closest_dist:
                        closest_dist = dist
                        closest_resource_tile = resource_tile
                if closest_resource_tile is not None:
                    actions.append(unit.move(unit.pos.direction_to(closest_resource_tile.pos)))
            elif(unit.can_build(game_state.map)):
                        unit.build_city()
                        print("city is built", file=sys.stderr)

            elif(len(player.units) >= len(player.cities)):
                # move to an empty tile to build a city
                destination = unit.pos
                while(not cell_is_free(game_state.map.get_cell_by_pos(destination))):
                    destination.y+=1
                move_dir = unit.pos.direction_to(destination)
                print("going to build a city", file=sys.stderr)
                actions.append(unit.move(move_dir))
            else:
                # if unit is a worker and there is no cargo space left, and we have cities, lets return to them
                if len(player.cities) > 0:
                    closest_dist = math.inf
                    closest_city_tile = None
                    for k, city in player.cities.items():
                        for city_tile in city.citytiles:
                            dist = city_tile.pos.distance_to(unit.pos)
                            if dist < closest_dist:
                                closest_dist = dist
                                closest_city_tile = city_tile
                    if closest_city_tile is not None:
                        move_dir = unit.pos.direction_to(closest_city_tile.pos)
                        actions.append(unit.move(move_dir))
    # you can add debug annotations using the functions in the annotate object
    # actions.append(annotate.circle(0, 0))
    
    return actions

In [ ]:
from agent import agent